In [0]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
%matplotlib inline

In [0]:
train_path = 'drive/My Drive/shoes/train'
valid_path = 'drive/My Drive/shoes/valid'
test_path = 'drive/My Drive/shoes/test'

In [0]:
if K.image_data_format()== 'channels_first':
  input_shape = (3,224,224)
else:
  input_shape = (224,224,3)

In [46]:
train_data_generator = ImageDataGenerator(rescale = 1. /225, shear_range = 0.2, zoom_range = 0.2, horizontal_flip=True)
valid_data_generator = ImageDataGenerator(rescale = 1. /255)

train_data = train_data_generator.flow_from_directory(train_path, target_size = (224,224), classes = ['adidas','nike','puma'], batch_size = 10)
valid_data = valid_data_generator.flow_from_directory(valid_path, target_size = (224,224), classes = ['adidas','nike','puma'], batch_size = 5)
test_data = ImageDataGenerator().flow_from_directory(test_path, target_size = (224,224), classes = ['adidas','nike','puma'], batch_size = 5)

Found 870 images belonging to 3 classes.
Found 105 images belonging to 3 classes.
Found 79 images belonging to 3 classes.


In [0]:
vgg16_model = keras.applications.vgg16.VGG16()

In [0]:
model = Sequential()
for layer in vgg16_model.layers:
  model.add(layer)

In [49]:
model.layers.pop()

In [50]:
for layer in model.layers:
  layer.trainable = False
  
model.add(Dense(3, activation = 'softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       295168    
__________

In [0]:
model.compile(Adam(lr= 0.0001), loss = 'categorical_crossentropy', metrics=['accuracy'])

In [0]:
model.fit_generator(train_data, steps_per_epoch= 87, validation_data = valid_data, validation_steps = 21, epochs = 50, verbose = 2)

Epoch 1/50
 - 512s - loss: 1.0986 - acc: 0.3172 - val_loss: 1.0986 - val_acc: 0.3238
Epoch 2/50
 - 506s - loss: 1.0986 - acc: 0.3345 - val_loss: 1.0986 - val_acc: 0.3143
Epoch 3/50
 - 504s - loss: 1.0986 - acc: 0.3264 - val_loss: 1.0985 - val_acc: 0.3048
Epoch 4/50
 - 505s - loss: 1.0985 - acc: 0.3701 - val_loss: 1.0985 - val_acc: 0.3429
Epoch 5/50
 - 504s - loss: 1.0985 - acc: 0.3345 - val_loss: 1.0985 - val_acc: 0.3524
Epoch 6/50


In [0]:
model.save_weights('vgg16_first_run.h5')

In [0]:
test_imgs, test_labels = next(test_data)
plots(test_imgs, titles = test_labels)

In [0]:
predictions = model.predict_generator(test_data, steps = 1, verbose = 0)

In [0]:
cm = confusion_matrix(test_labels, predictions[:,0])

In [0]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plot_confusion_matrix(y_test, y_pred, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plot_confusion_matrix(y_test, y_pred, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

In [0]:
cm_plot_labels = ['adidas','nike','puma']
plot_confusion_matrix(cm, cm_plot_labels, title = 'Confusion Matrix')

In [0]:
img_pred = image.load_img('testimage1.jpg', target_size = (150,150))
img_pred = image.img_to_array(img_pred)
img_pred = np.expand_dims(img_pred ,axis = 0)

result = model.predict(img_pred)
print(result)

img_pred = image.load_img('testimage2.jpg', target_size = (150,150))
img_pred = image.img_to_array(img_pred)
img_pred = np.expand_dims(img_pred ,axis = 0)

result = model.predict(img_pred)
print(result)

img_pred = image.load_img('testimage3.jpg', target_size = (150,150))
img_pred = image.img_to_array(img_pred)
img_pred = np.expand_dims(img_pred ,axis = 0)

result = model.predict(img_pred)
print(result)